<a href="https://colab.research.google.com/github/rvp3011/Lab0/blob/main/Practtice7_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score

In [ ]:
ames = pd.read_csv("AmesHousing.csv")

good_cols = ames.isna().sum() < 100
ames = ames.loc[:,good_cols]

ames = ames.dropna()

In [ ]:
X = ames.drop(["SalePrice", "Order", "PID"], axis = 1)
y = ames["SalePrice"]


ct = ColumnTransformer(
  [
    ("dummify",
    OneHotEncoder(sparse_output = False, handle_unknown='ignore'),
    make_column_selector(dtype_include=object)),
    ("standardize",
    StandardScaler(),
    make_column_selector(dtype_include=np.number))
  ],
  remainder = "passthrough"
)

lr_pipeline_1 = Pipeline(
  [("preprocessing", ct),
  ("linear_regression", LinearRegression())]
)

In [36]:
ridge_pipeline_1 = Pipeline(
  [("preprocessing", ct),
  ("ridge_regression", Ridge(alpha=1))]
)

In [38]:
X = X.dropna()
y = y.loc[X.index]

In [41]:
ames = pd.read_csv("AmesHousing.csv")
ames = ames.dropna()

X = ames.drop(["SalePrice", "Order", "PID"], axis = 1)
y = ames["SalePrice"]

In [53]:
ames = pd.read_csv("AmesHousing.csv")

good_cols = ames.isna().sum() < 100
ames = ames.loc[:,good_cols]

X = ames.drop(["SalePrice", "Order", "PID"], axis = 1)
y = ames["SalePrice"]


In [46]:
from sklearn.impute import SimpleImputer

ridge_pipeline_1 = Pipeline(
  [("preprocessing", ct),
   ("imputer", SimpleImputer(strategy='mean')),
  ("ridge_regression", Ridge(alpha=1))]
)

ridge_cv_scores = cross_val_score(ridge_pipeline_1, X, y, cv=5)
display(ridge_cv_scores)

array([0.90036389, 0.91716568, 0.80433812, 0.79407721, 0.9188616 ])

In [48]:
from sklearn.impute import SimpleImputer

lr_pipeline_1 = Pipeline(
  [("preprocessing", ct),
   ("imputer", SimpleImputer(strategy='mean')),
  ("linear_regression", LinearRegression())]
)

lr_cv_scores = cross_val_score(lr_pipeline_1, X, y, cv=5)
display(lr_cv_scores)

array([0.89835654, 0.90922977, 0.79663797, 0.78055196, 0.91162054])

In [52]:


mean_ridge_score = np.mean(ridge_cv_scores)
mean_lr_score = np.mean(lr_cv_scores)

print(mean_ridge_score)
print(mean_lr_score)

0.8669612970902986
0.8592793562344443


In [54]:
from sklearn.linear_model import Lasso

lasso_pipeline_1 = Pipeline(
    [("preprocessing", ct),
     ("imputer", SimpleImputer(strategy='mean')),
     ("lasso_regression", Lasso(alpha=1.0))]
)

In [59]:
from sklearn.model_selection import GridSearchCV

param_grid = {'lasso_regression__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

lasso_grid_search = GridSearchCV(lasso_pipeline_1, param_grid, cv=5)

lasso_grid_search.fit(X, y)

best_alpha = lasso_grid_search.best_params_['lasso_regression__alpha']
print(best_alpha)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.334e+11, tolerance: 1.414e+09
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.531e+11, tolerance: 1.534e+09
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.892e+11, tolerance: 1.527e

The best alpha value is: 100


In [56]:
best_lasso_pipeline = Pipeline(
  [("preprocessing", ct),
   ("imputer", SimpleImputer(strategy='mean')),
  ("lasso_regression", Lasso(alpha=best_alpha))]
)

best_lasso_pipeline.fit(X, y)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('dummify',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7ddb952a6ae0>),
                                                 ('standardize',
                                                  StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7ddb93e31e20>)])),
                ('imputer', SimpleImputer()),
                ('lasso_regression', Lasso(alpha=100))])

In [58]:
lr_pipeline_1.fit(X, y)
ridge_pipeline_1.fit(X, y)

lr_coefficients = lr_pipeline_1.named_steps['linear_regression'].coef_
ridge_coefficients = ridge_pipeline_1.named_steps['ridge_regression'].coef_
lasso_coefficients = best_lasso_pipeline.named_steps['lasso_regression'].coef_

X_sample = X.head(1)
ct.fit(X, y)
X_transformed_sample = ct.transform(X_sample)

feature_names = ct.get_feature_names_out()

coefficients_df = pd.DataFrame({
    'Feature': feature_names,
    'Linear Regression': lr_coefficients,
    'Ridge Regression': ridge_coefficients,
    'Lasso Regression': lasso_coefficients
})

coefficients_df['Linear_Abs'] = coefficients_df['Linear Regression'].abs()
coefficients_df = coefficients_df.sort_values(by='Linear_Abs', ascending=False).drop('Linear_Abs', axis=1)

display(coefficients_df)

,Feature,Linear Regression,Ridge Regression,Lasso Regression
92,dummify__Roof Matl_ClyTile,-554222.075632,-232493.077784,-214297.391241
99,dummify__Roof Matl_WdShngl,121128.901132,84742.205175,22819.415541
69,dummify__Condition 2_PosN,-112642.009597,-77843.552800,-11171.143964
94,dummify__Roof Matl_Membran,111776.923622,30073.177599,0.000000
38,dummify__Neighborhood_GrnHill,104080.982266,70456.733426,0.000000
...,...,...,...,...
11,dummify__Lot Shape_IR3,176.806857,-10609.086121,-0.000000
129,dummify__Exterior 2nd_Stucco,-146.229454,-3566.718937,-0.000000
228,dummify__Sale Condition_Normal,-125.356531,-168.074655,0.000000
217,dummify__Sale Type_ConLD,77.550702,714.171564,0.000000


In [60]:
from sklearn.linear_model import ElasticNet

enet_pipeline_1 = Pipeline(
    [("preprocessing", ct),
     ("imputer", SimpleImputer(strategy='mean')),
     ("elastic_net", ElasticNet())]
)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'elastic_net__alpha': [0.001, 0.01, 0.1, 1, 10, 100],
    'elastic_net__l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]
}

enet_grid_search = GridSearchCV(enet_pipeline_1, param_grid, cv=5, n_jobs=-1)

enet_grid_search.fit(X, y)

print(enet_grid_search.best_params_)

In [62]:
best_alpha_enet = enet_grid_search.best_params_['elastic_net__alpha']
best_l1_ratio_enet = enet_grid_search.best_params_['elastic_net__l1_ratio']

best_enet_pipeline = Pipeline(
  [("preprocessing", ct),
   ("imputer", SimpleImputer(strategy='mean')),
  ("elastic_net", ElasticNet(alpha=best_alpha_enet, l1_ratio=best_l1_ratio_enet))]
)

best_enet_pipeline.fit(X, y)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('dummify',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7ddb952a6ae0>),
                                                 ('standardize',
                                                  StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7ddb93e31e20>)])),
                ('imputer', SimpleImputer()),
                ('elastic_net', ElasticNet(alpha=100, l1_ratio=1))])

In [63]:
enet_coefficients = best_enet_pipeline.named_steps['elastic_net'].coef_

coefficients_df = pd.DataFrame({
    'Feature': feature_names,
    'Linear Regression': lr_coefficients,
    'Ridge Regression': ridge_coefficients,
    'Lasso Regression': lasso_coefficients,
    'Elastic Net': enet_coefficients
})

coefficients_df['Linear_Abs'] = coefficients_df['Linear Regression'].abs()
coefficients_df = coefficients_df.sort_values(by='Linear_Abs', ascending=False).drop('Linear_Abs', axis=1)

display(coefficients_df)

,Feature,Linear Regression,Ridge Regression,Lasso Regression,Elastic Net
92,dummify__Roof Matl_ClyTile,-554222.075632,-232493.077784,-214297.391241,-214297.391241
99,dummify__Roof Matl_WdShngl,121128.901132,84742.205175,22819.415541,22819.415541
69,dummify__Condition 2_PosN,-112642.009597,-77843.552800,-11171.143964,-11171.143964
94,dummify__Roof Matl_Membran,111776.923622,30073.177599,0.000000,0.000000
38,dummify__Neighborhood_GrnHill,104080.982266,70456.733426,0.000000,0.000000
...,...,...,...,...,...
11,dummify__Lot Shape_IR3,176.806857,-10609.086121,-0.000000,-0.000000
129,dummify__Exterior 2nd_Stucco,-146.229454,-3566.718937,-0.000000,-0.000000
228,dummify__Sale Condition_Normal,-125.356531,-168.074655,0.000000,0.000000
217,dummify__Sale Type_ConLD,77.550702,714.171564,0.000000,0.000000
